In [1]:
# --- BLOCK 1: Setup, Downloads, and Base Configuration ---

# 1. Install necessary libraries
print("--- Installing required libraries... ---")
!pip install opencv-python gdown numpy reportlab==3.6.13 # Specific version for compatibility
!pip install ultralytics # For YOLOv8
print("--- Libraries installed. ---")

import cv2
import os
import shutil
import gdown
import numpy as np
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from ultralytics import YOLO # For object detection (YOLOv8)
import logging

# Configure logging for better feedback
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Configuration ---
VIDEO_URL = "https://drive.google.com/uc?id=1f9Dk2SAGaX8-7-qBai14WydAiAeeAnY3"
VIDEO_FILENAME = "raw_video.mp4"
OUTPUT_BASE_DIR = "Processed_Video"
TRAIN_NUMBER = "12309" # As per the assignment example

# Object Detection Model Configuration
# IMPORTANT: For real-world door detection, you would need a custom-trained model.
# This uses a generic YOLOv8n model as a placeholder.
# You might need to download 'yolov8n.pt' or your custom trained model to a path accessible by Colab.
YOLO_MODEL_PATH = 'yolov8n.pt' # Using nano model as a placeholder.
# If you have a custom trained model for doors, e.g., 'door_detector.pt', use that path.
# Example: YOLO_MODEL_PATH = '/content/drive/MyDrive/yolo_models/door_detector.pt'

# --- 2. Download the raw video file from Google Drive ---
logger.info(f"--- Downloading the raw video file from {VIDEO_URL}... ---")
try:
    gdown.download(VIDEO_URL, VIDEO_FILENAME, quiet=False)
    video_path = VIDEO_FILENAME
    logger.info("Video downloaded successfully.")
except Exception as e:
    logger.error(f"Error downloading video: {e}")
    video_path = None

if not video_path or not os.path.exists(video_path):
    logger.critical("Video file not found. Please ensure the URL is correct and accessible. Exiting.")
    # In a real script, you might raise an exception or sys.exit() here.
    # For Colab, we'll proceed with a dummy path to avoid crashing the whole notebook.
    video_path = "dummy_video.mp4" # Ensure this won't actually be opened later.

# Create main output directory
os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)
logger.info(f"Created base output directory: {OUTPUT_BASE_DIR}")

# Prepare directories for report and individual coaches
processed_video_base = os.path.join(OUTPUT_BASE_DIR, "individual_coaches")
report_output_dir = os.path.join(OUTPUT_BASE_DIR, "Final Report---Assignment Submission")
os.makedirs(processed_video_base, exist_ok=True)
os.makedirs(report_output_dir, exist_ok=True)
logger.info(f"Created subdirectories: {processed_video_base} and {report_output_dir}")

# Global list to store data for the final report
report_data = []

# Placeholder for YOLO model (loaded once)
yolo_model = None
try:
    yolo_model = YOLO(YOLO_MODEL_PATH)
    logger.info(f"YOLOv8 model loaded from {YOLO_MODEL_PATH}")
except Exception as e:
    logger.warning(f"Could not load YOLOv8 model from {YOLO_MODEL_PATH}. Object detection will be skipped. Error: {e}")
    logger.warning("Please ensure 'yolov8n.pt' is available or specify your custom model path correctly.")

--- Installing required libraries... ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for reportlab: filename=reportlab-3.6.13-cp312-cp312-linux_x86_64.whl size=2237988 sha256=6c62303def098e49e581f1832eb9c16a09e66e4d5a91f1ac29bccb7a958ca052
  Stored in directory: /root/.cache/pip/wheels/48/67/58/d54706b91458551310cfa11bd10777cb55438c6f33bf3a5292
Successfully built reportlab
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.2 MB/s eta 0:00:00
--- Libraries installed. ---
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Downloading...
From: https://drive.google.com/uc?id=1f9Dk2SAGaX8-7-qBai14WydAiAeeAnY3
To: /content/raw_video.mp4
100%|██████████| 79.2M/79.2M [00:00<00:00, 91.6MB/s]


In [2]:
# --- BLOCK 2: Video Processing - Splitting and Frame Extraction (Enhanced) ---

logger.info("\n--- Starting video processing and coach/engine segmentation... ---")

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    logger.error("Error: Could not open video file. Please check if the video downloaded correctly.")
    # Exit gracefully if video can't be opened
    report_data.append({"type": "summary", "text": "Video could not be processed due to file error."})
else:
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames_in_video = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for MP4

    coach_count = 0
    engine_count = 0
    current_wagon_writer = None
    is_between_wagons = True # Flag to indicate if we are in a gap or processing a wagon
    prev_frame_gray = None # Storing previous frame for differencing

    # Store all frames for the current wagon to allow for full coverage image selection
    current_wagon_frames = []
    current_wagon_name = ""
    current_wagon_dir = ""

    frame_idx = 0
    logger.info(f"Processing video: {VIDEO_FILENAME} with {total_frames_in_video} frames at {fps} FPS.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break # End of video

        frame_idx += 1
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame_gray is None:
            prev_frame_gray = gray_frame
            continue # Skip first frame as there's no previous frame to compare

        # --- Core Detection Algorithm: Frame Differencing ---
        # Calculate absolute difference between current and previous grayscale frame
        frame_diff = cv2.absdiff(prev_frame_gray, gray_frame)

        # Threshold the difference image to highlight significant changes
        # Pixels with intensity difference > 30 become white (255), others black (0)
        _, thresh = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)

        # Calculate the percentage of non-zero pixels (changed pixels)
        change_percentage = (np.count_nonzero(thresh) / (frame_width * frame_height)) * 100

        # --- Color Analysis for Engine/Wagon Differentiation (Heuristic) ---
        # This is a simple heuristic. For robust detection, a trained object detector is better.
        # Assuming engines in *this specific video* have a distinct color profile (e.g., blue, red, or very dark)
        mean_color_bgr = np.mean(frame, axis=(0, 1))
        # Example: Assume engines are very dark or have a specific dominant color not common in coaches
        # Adjust these thresholds based on visual inspection of your video's engine colors.
        is_engine_heuristic = False
        if (mean_color_bgr[0] > 100 and mean_color_bgr[1] < 80 and mean_color_bgr[2] < 80) or \
           (mean_color_bgr[0] < 80 and mean_color_bgr[1] < 80 and mean_color_bgr[2] > 100) or \
           (np.mean(mean_color_bgr) < 50): # Very dark for example
            is_engine_heuristic = True

        # --- Wagon/Engine Detection Logic ---
        # Condition 1: Transition from a gap to a new wagon/engine
        if is_between_wagons and change_percentage < 5: # If change is low, a solid object is passing
            if is_engine_heuristic and engine_count < 2: # Assuming max 2 engines from "Points to Ponder"
                engine_count += 1
                wagon_type = "engine"
                wagon_counter = engine_count
            else:
                coach_count += 1
                wagon_type = "wagon"
                wagon_counter = coach_count

            is_between_wagons = False # We are now inside a wagon

            current_wagon_name = f"{wagon_type}_{wagon_counter}"
            current_wagon_dir = os.path.join(processed_video_base, f"{TRAIN_NUMBER}_{current_wagon_name}")
            os.makedirs(current_wagon_dir, exist_ok=True)
            logger.info(f"Detected {current_wagon_name}. Created directory: {current_wagon_dir}")

            # Initialize video writer for the current wagon
            wagon_video_path = os.path.join(current_wagon_dir, f"{TRAIN_NUMBER}_{current_wagon_name}.mp4")
            current_wagon_writer = cv2.VideoWriter(wagon_video_path, fourcc, fps, (frame_width, frame_height))
            current_wagon_frames = [] # Reset frames for the new wagon

            logger.info(f"Started recording video for {current_wagon_name}.")

        # Condition 2: Transition from a wagon/engine to a gap
        elif not is_between_wagons and change_percentage > 10: # If change is high, we've hit a gap
            is_between_wagons = True # We are now in a gap

            # Release the video writer for the completed wagon
            if current_wagon_writer is not None:
                current_wagon_writer.release()
                logger.info(f"Finished recording video for {current_wagon_name}.")

                # --- Process collected frames for the completed wagon ---
                # Add data for report (this will be used by the reporting function)
                report_data.append({
                    "type": "wagon_data",
                    "wagon_name": current_wagon_name,
                    "wagon_type": current_wagon_name.split('_')[0],
                    "video_path": wagon_video_path,
                    "frames_list": list(current_wagon_frames) # Store a copy of collected frames
                })
                current_wagon_frames = [] # Clear frames after processing

        # While inside a wagon, continue writing frames and collecting them
        if not is_between_wagons and current_wagon_writer is not None:
            current_wagon_writer.write(frame)
            current_wagon_frames.append(frame.copy()) # Store a copy of the frame

        prev_frame_gray = gray_frame

    # Release any remaining resources after the loop
    cap.release()
    if current_wagon_writer is not None and not is_between_wagons:
        current_wagon_writer.release()
        logger.info(f"Finished recording video for {current_wagon_name} (end of video).")
        # Ensure last wagon's data is added if loop ended while inside a wagon
        report_data.append({
            "type": "wagon_data",
            "wagon_name": current_wagon_name,
            "wagon_type": current_wagon_name.split('_')[0],
            "video_path": wagon_video_path,
            "frames_list": list(current_wagon_frames)
        })

    logger.info("--- Video processing complete. ---")
    logger.info(f"Total coaches detected: {coach_count}")
    logger.info(f"Total engines detected: {engine_count}")

In [3]:
# --- BLOCK 3: Full Coverage Image Extraction & Object Detection (Door Detection) ---

logger.info("\n--- Starting full coverage image extraction and object detection... ---")

def get_full_coverage_images(frames, wagon_name, images_dir, min_coverage_images=5, threshold_similarity=0.4):
    """
    Extracts a minimal set of images for full wagon coverage using ORB features
    and saves them.
    Returns paths to saved images.
    """
    if not frames:
        logger.warning(f"No frames available for {wagon_name}.")
        return []

    saved_image_paths = []

    # Initialize ORB detector
    orb = cv2.ORB_create()

    # Take the first frame as the starting point for coverage
    coverage_images = [frames[0]]
    last_kp, last_des = orb.detectAndCompute(cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY), None)

    img_counter = 1
    image_filename = os.path.join(images_dir, f"{TRAIN_NUMBER}_{wagon_name}_{img_counter}.jpg")
    cv2.imwrite(image_filename, frames[0])
    saved_image_paths.append(image_filename)
    img_counter += 1

    for i in range(1, len(frames)):
        current_frame = frames[i]
        current_kp, current_des = orb.detectAndCompute(cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY), None)

        if last_des is None or current_des is None or len(last_des) < 2 or len(current_des) < 2:
            # Not enough features, just add after a certain number of frames or if enough images not yet collected
            if len(coverage_images) < min_coverage_images and i % (len(frames) // min_coverage_images + 1) == 0:
                 coverage_images.append(current_frame)
                 last_kp, last_des = current_kp, current_des
                 image_filename = os.path.join(images_dir, f"{TRAIN_NUMBER}_{wagon_name}_{img_counter}.jpg")
                 cv2.imwrite(image_filename, current_frame)
                 saved_image_paths.append(image_filename)
                 img_counter += 1
            continue

        # Use Brute-Force Matcher with Hamming distance for ORB
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = matcher.match(last_des, current_des)

        # Sort matches by distance
        matches = sorted(matches, key=lambda x: x.distance)

        # Calculate similarity based on number of good matches
        # This threshold determines how 'different' a new frame must be to be included
        similarity = len(matches) / min(len(last_des), len(current_des)) if min(len(last_des), len(current_des)) > 0 else 0

        # If similarity is below threshold, or if we haven't collected enough images yet
        if similarity < threshold_similarity or (len(coverage_images) < min_coverage_images and i % (len(frames) // min_coverage_images + 1) == 0):
            coverage_images.append(current_frame)
            last_kp, last_des = current_kp, current_des

            image_filename = os.path.join(images_dir, f"{TRAIN_NUMBER}_{wagon_name}_{img_counter}.jpg")
            cv2.imwrite(image_filename, current_frame)
            saved_image_paths.append(image_filename)
            img_counter += 1
            logger.debug(f"Added frame {img_counter-1} for {wagon_name} (similarity: {similarity:.2f})")

    logger.info(f"Extracted {len(saved_image_paths)} full coverage images for {wagon_name}.")
    return saved_image_paths

# --- Component Detection (Doors - Conceptual YOLOv8 Integration) ---
def detect_doors_and_state(image_path, yolo_model):
    """
    Performs object detection on an image to find doors and their state (open/closed).
    Returns a list of detected objects (class_name, bbox, confidence, state).
    """
    detected_objects = []
    if yolo_model is None:
        logger.warning(f"YOLOv8 model not loaded. Skipping object detection for {image_path}.")
        return []

    try:
        results = yolo_model(image_path) # Run inference

        # Placeholder for custom trained model classes
        # If your model has 'door', 'door_open', 'door_closed' classes, adapt this.
        # Example for a generic model: let's assume class '0' is 'person' and we pretend it's a door.
        # For a real door detector, you would map results[0].names to your custom classes.
        class_names = results[0].names # Get class names from the model

        for r in results:
            for box in r.boxes:
                class_id = int(box.cls)
                class_name = class_names[class_id]
                conf = float(box.conf)
                x1, y1, x2, y2 = map(int, box.xyxy[0]) # Bounding box coordinates

                # --- Door State Classification (Conceptual) ---
                # This part is highly dependent on a custom-trained model for 'door_open'/'door_closed'
                # or a very clever heuristic based on image content within the bounding box.
                # Here, we'll just simulate a state.
                door_state = "unknown"
                # If your model detects specific 'door_open' or 'door_closed' classes:
                if "door" in class_name.lower(): # Generic door
                     # Simulate state based on bounding box size or region analysis (highly heuristic)
                    if (x2 - x1) * (y2 - y1) < 1000: # Very small 'door'
                        door_state = "closed" # Placeholder heuristic
                    else:
                        door_state = "open" # Placeholder heuristic
                    # This needs real ML classification or a robust heuristic.

                detected_objects.append({
                    "class_name": class_name,
                    "bbox": (x1, y1, x2, y2),
                    "confidence": conf,
                    "state": door_state # Only relevant if it's a door
                })
    except Exception as e:
        logger.error(f"Error during object detection for {image_path}: {e}")

    return detected_objects

# Iterate through the collected wagon data for image extraction and object detection
for item in report_data:
    if item["type"] == "wagon_data":
        wagon_name = item["wagon_name"]
        wagon_frames = item["frames_list"]

        wagon_main_dir = os.path.join(processed_video_base, f"{TRAIN_NUMBER}_{wagon_name}")
        images_output_dir = os.path.join(wagon_main_dir, "extracted_frames")
        os.makedirs(images_output_dir, exist_ok=True)

        logger.info(f"Processing frames for {wagon_name} to get full coverage and detect components...")

        # Get full coverage images
        full_coverage_image_paths = get_full_coverage_images(
            wagon_frames, wagon_name, images_output_dir, min_coverage_images=5, threshold_similarity=0.4
        )
        item["full_coverage_images"] = full_coverage_image_paths

        # Perform object detection on each full coverage image
        item["detected_components_by_image"] = {}
        door_open_count = 0
        door_closed_count = 0

        for img_path in full_coverage_image_paths:
            # Read image, perform detection, and annotate
            img = cv2.imread(img_path)
            detected_components = detect_doors_and_state(img_path, yolo_model)

            annotated_img = img.copy()
            door_found_in_image = False

            for comp in detected_components:
                x1, y1, x2, y2 = comp["bbox"]
                class_name = comp["class_name"]
                conf = comp["confidence"]
                state = comp["state"]

                # Annotate image
                color = (0, 255, 0) # Green for general detection
                text_color = (255, 255, 255) # White text
                if "door" in class_name.lower():
                    door_found_in_image = True
                    if state == "open":
                        color = (0, 0, 255) # Red for open door
                        door_open_count += 1
                    elif state == "closed":
                        color = (255, 0, 0) # Blue for closed door
                        door_closed_count += 1
                    cv2.rectangle(annotated_img, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(annotated_img, f"{class_name} ({state}) {conf:.2f}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                else: # For other detected objects by generic YOLO
                    cv2.rectangle(annotated_img, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(annotated_img, f"{class_name} {conf:.2f}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # Save the annotated image
            annotated_img_path = img_path.replace(".jpg", "_annotated.jpg")
            cv2.imwrite(annotated_img_path, annotated_img)
            item["detected_components_by_image"][img_path] = {
                "annotated_image_path": annotated_img_path,
                "detections": detected_components,
                "door_found": door_found_in_image
            }

        # Aggregate door counts (deduplicate if same door detected across multiple frames of one wagon)
        # This is complex and might need a separate tracking algorithm. For simplicity, just sum per image for report.
        item["total_doors_open_in_wagon"] = door_open_count
        item["total_doors_closed_in_wagon"] = door_closed_count
        item["door_open_status_flag"] = "Yes" if door_open_count > 0 else "No"

logger.info("--- Full coverage image extraction and object detection complete. ---")


image 1/1 /content/Processed_Video/individual_coaches/12309_wagon_1/extracted_frames/12309_wagon_1_1.jpg: 288x640 1 train, 1 truck, 276.8ms
Speed: 7.7ms preprocess, 276.8ms inference, 33.1ms postprocess per image at shape (1, 3, 288, 640)

image 1/1 /content/Processed_Video/individual_coaches/12309_wagon_1/extracted_frames/12309_wagon_1_2.jpg: 288x640 1 bus, 1 train, 1 truck, 104.7ms
Speed: 2.1ms preprocess, 104.7ms inference, 2.1ms postprocess per image at shape (1, 3, 288, 640)

image 1/1 /content/Processed_Video/individual_coaches/12309_wagon_1/extracted_frames/12309_wagon_1_3.jpg: 288x640 2 trains, 114.5ms
Speed: 2.1ms preprocess, 114.5ms inference, 1.9ms postprocess per image at shape (1, 3, 288, 640)

image 1/1 /content/Processed_Video/individual_coaches/12309_wagon_1/extracted_frames/12309_wagon_1_4.jpg: 288x640 2 trains, 103.1ms
Speed: 2.0ms preprocess, 103.1ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 640)

image 1/1 /content/Processed_Video/individual_coach

In [6]:
import os
import shutil
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
import logging

# Configure logging for better feedback
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

logger.info("\n--- Generating PDF report... ---")

def generate_pdf_report(report_data, output_filepath):
    doc = SimpleDocTemplate(output_filepath, pagesize=A4)
    styles = getSampleStyleSheet()

    # Custom style for summary text
    summary_style = ParagraphStyle(
        name='SummaryStyle',
        parent=styles['Normal'],
        fontSize=12,
        leading=16,
        spaceAfter=12
    )

    # FIX: Define a custom 'Small' style to replace the one that doesn't exist
    small_style = ParagraphStyle(
        name='SmallStyle',
        parent=styles['Normal'],
        fontSize=8,
        leading=10,
        spaceAfter=6
    )

    flowables = []

    # --- First Page: Summary Report ---
    flowables.append(Paragraph("<b>Train Side View Analysis Report</b>", styles['h1']))
    flowables.append(Spacer(1, 0.2 * inch))

    # General Summary
    total_wagons = sum(1 for item in report_data if item.get("wagon_type") == "wagon")
    total_engines = sum(1 for item in report_data if item.get("wagon_type") == "engine")

    flowables.append(Paragraph(f"<b>Summary Overview:</b>", styles['h2']))
    flowables.append(Paragraph(f"Train Number: <b>{TRAIN_NUMBER}</b>", summary_style))
    flowables.append(Paragraph(f"Total Engines Detected: <b>{total_engines}</b>", summary_style))
    flowables.append(Paragraph(f"Total Wagons (Coaches) Detected: <b>{total_wagons}</b>", summary_style))
    flowables.append(Spacer(1, 0.2 * inch))

    # CCTV Specific Information (placeholder counts)
    total_doors_open_overall = sum(item.get("total_doors_open_in_wagon", 0) for item in report_data if item["type"] == "wagon_data")
    total_doors_closed_overall = sum(item.get("total_doors_closed_in_wagon", 0) for item in report_data if item["type"] == "wagon_data")

    flowables.append(Paragraph(f"<b>CCTV Overview:</b>", styles['h2']))
    flowables.append(Paragraph(f"Door of wagon Open in this rack: <b>{'Yes' if total_doors_open_overall > 0 else 'No'}</b>", summary_style))
    flowables.append(Paragraph(f"Count of Door of wagon Open: <b>{total_doors_open_overall}</b>", summary_style))

    wagon_index_from_engine = []
    current_wagon_from_engine_idx = 0
    for item in report_data:
        if item.get("type") == "wagon_data":
            if item.get("wagon_type") == "engine":
                current_wagon_from_engine_idx = 0
            else:
                current_wagon_from_engine_idx += 1
                wagon_index_from_engine.append(str(current_wagon_from_engine_idx))
    flowables.append(Paragraph(f"Count Wagon from Engine: <b>{', '.join(wagon_index_from_engine)}</b>", summary_style))

    flowables.append(PageBreak())

    # --- Details for Each Wagon/Engine ---
    wagon_report_counter = 0
    for item in report_data:
        if item.get("type") == "wagon_data":
            wagon_report_counter += 1
            wagon_name = item.get("wagon_name", "N/A")
            wagon_type = item.get("wagon_type", "N/A")

            flowables.append(Paragraph(f"<b>{wagon_name.replace('_', ' ').title()} Details:</b>", styles['h2']))
            flowables.append(Paragraph(f"Type: {wagon_type.title()}", styles['Normal']))
            flowables.append(Paragraph(f"Individual Video: {item.get('video_path', 'N/A')}", styles['Normal']))

            if wagon_type == "wagon":
                flowables.append(Paragraph(f"Door Open Status: <b>{item.get('door_open_status_flag', 'N/A')}</b>", styles['Normal']))
                flowables.append(Paragraph(f"Doors Detected (Open): {item.get('total_doors_open_in_wagon', 0)}", styles['Normal']))
                flowables.append(Paragraph(f"Doors Detected (Closed): {item.get('total_doors_closed_in_wagon', 0)}", styles['Normal']))

            flowables.append(Spacer(1, 0.1 * inch))

            flowables.append(Paragraph("<b>Full Coverage Images:</b>", styles['h3']))
            images_for_this_wagon = item.get("full_coverage_images", [])
            if images_for_this_wagon:
                for img_path in images_for_this_wagon:
                    annotated_path = item["detected_components_by_image"].get(img_path, {}).get("annotated_image_path", None)
                    if annotated_path and os.path.exists(annotated_path):
                        img = Image(annotated_path, width=4*inch, height=3*inch)
                        flowables.append(img)
                        flowables.append(Paragraph(f"Image: {os.path.basename(annotated_path)}", small_style))
                        detections_for_image = item["detected_components_by_image"][img_path].get("detections", [])
                        if detections_for_image:
                            for det in detections_for_image:
                                flowables.append(Paragraph(f"- Detected: {det['class_name']} ({det['state']}) Conf: {det['confidence']:.2f}", small_style))
                        flowables.append(Spacer(1, 0.1 * inch))
                    else:
                        flowables.append(Paragraph(f"Could not load image: {os.path.basename(img_path)} or its annotation.", small_style))
            else:
                flowables.append(Paragraph("No full coverage images extracted for this wagon.", styles['Normal']))

            if wagon_report_counter < (total_wagons + total_engines):
                flowables.append(PageBreak())

    try:
        doc.build(flowables)
        logger.info(f"PDF report generated successfully: {output_filepath}")
    except Exception as e:
        logger.error(f"Error generating PDF: {e}")

pdf_report_path = os.path.join(report_output_dir, f"{TRAIN_NUMBER}_Train_Analysis_Report.pdf")
generate_pdf_report(report_data, pdf_report_path)

# --- Final Archiving ---
try:
    shutil.make_archive(OUTPUT_BASE_DIR, 'zip', OUTPUT_BASE_DIR)
    logger.info(f"Output compressed into '{OUTPUT_BASE_DIR}.zip'.")
except Exception as e:
    logger.error(f"Error zipping output folder: {e}")

logger.info("\n--- Processing complete! ---")
logger.info(f"You can now download '{OUTPUT_BASE_DIR}.zip' from the Colab file explorer (left sidebar).")
logger.info("\n--- Important Next Steps for Submission ---")
logger.info("1. Download the 'Processed_Video.zip' file from Colab.")
logger.info("2. Upload the unzipped contents to your Google Drive as specified.")
logger.info("3. Create your screen recording explaining the project.")
logger.info("4. Write your README.md file (setup, features, limitations).")